# JSON Schema Form's UI Schema
One quirk of [react-jsonschema-form](https://react-jsonschema-form.readthedocs.io) is it's `uiSchema`,
which ironically is _not_ constrained by a JSON schema.

Since it _is_ unconstrained, we can add whatever we like: in this case, a schema for `uiSchema` that could generate part of a applies `@detahbeds/jupyterlab-rjsf`'s CodeMirror mode.

In [152]:
!conda install -yc conda-forge requests_cache

Solving environment: done

## Package Plan ##

  environment location: /home/weg/projects/deathbeds/wxyz/envs/default

  added / updated specs:
    - requests_cache


The following NEW packages will be INSTALLED:

  requests_cache     conda-forge/noarch::requests_cache-0.4.13-py_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [169]:
import requests, lxml.etree as etree, requests_cache, bleach
requests_cache.install_cache("codemirror-schema-form")

In [180]:
doc = etree.HTML(requests.get("https://codemirror.net/doc/manual.html#config").text)
clean = lambda x: x if x in ['string', 'boolean'] else 'boolean' if x in ['boolean'] else None
easy_cm_options = {
    e.text: {
        "description": bleach.clean(etree.tostring(e.getparent()
            .getparent()
            .getnext()).decode("utf-8").replace('<dd>', '').replace('</dd>', '')),
        "type": [
            clean(arg.strip()) 
            for arg in etree
                .tostring(e.getparent())
                .decode("utf-8")
                .replace("</code>", "")
                .split(":")[1]
                .split('|')
            if arg.strip() in ['string', 'boolean', 'bool']
        ]
    } 
    for e in doc.xpath("""//*[@id='config']//strong""")
}
easy_cm_options = {
    k: v
    for k, v in easy_cm_options.items()
    if v["type"]
}
easy_cm_options

{'value': {'description': 'The starting value of the editor. Can be a string, or\n      a <a href="#api_doc">document object</a>.\n\n      ',
  'type': ['string']},
 'mode': {'description': 'The mode to use. When not given, this will default to the\n      first mode that was loaded. It may be a string, which either\n      simply names the mode or is\n      a <a href="http://en.wikipedia.org/wiki/MIME">MIME</a> type\n      associated with the mode. The value <code>"null"</code>\n      indicates no highlighting should be applied. Alternatively, it\n      may be an object containing configuration options for the mode,\n      with a <code>name</code> property that names the mode (for\n      example <code>{name: "javascript", json: true}</code>). The demo\n      pages for each mode contain information about what configuration\n      parameters the mode supports. You can ask CodeMirror which modes\n      and MIME types have been defined by inspecting\n      the <code>CodeMirror.modes</code>\

In [181]:
cm_schema = {
    "type": "object", 
    "title": "CodeMirror Options",
    "description": "CodeMirror JSON properties to initialize inside a JSON Schema Form https://codemirror.net/doc/manual.html#config",
    "properties": {
         # TODO: automate discovery
        "theme": {
            "type": "string",
            "enum": ["zenburn", "the-matrix", "material"],
            "default": "zenburn"
        },
        "lineNumbers": {
            "type": "boolean",
            "default": True
        },
        "spellcheck": {
            "type": "boolean",
            "default": True
        },
        "inputStyle": {
            "type": "string",
            "enum": ["contenteditable", "textarea"],
            "default": "contenteditable"
        },
        **easy_cm_options
    }
}

In [182]:
with __import__("importnb").Notebook():
    import JSON_Schema_Form

In [183]:
cm_schema_form = JSON_Schema_Form.JSONSchemaForm(schema=cm_schema)
cm_schema_form

JSONSchemaForm(schema={'type': 'object', 'title': 'CodeMirror Options', 'description': 'CodeMirror JSON proper…

In [33]:
box.children += tuple([cm_schema_form])

In [ ]:
%%html
<style>
.jp-SchemaForm > .form-group.field.field-object {
    display: flex;
    flex-direction: column;
}
.form-group.field {
    display: flex;
    flex: 1;
    flex-direction: row;
    max-width: 25%;
}
.form-group.field > * {
    flex: 1;
}
fieldset {
    display: flex;
    flex-direction: row;
    flex-wrap: wrap;
}
</style>